In [23]:
import glob
import mne
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt

In [24]:
files_eeg_b= glob.glob(r"C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_*_b_*.csv", recursive= False)
files_eeg_q = glob.glob(r"C:\Users\yehuda\pain_of_payment\Data_PoP\mindMonitor_*_q_*.csv", recursive= False)

In [31]:
'''
test to check with one file
dont need to run it - just for debbugging
'''
freq_bands = dict(delta=(0,4), theta=(4,8), alpha=(8,12), beta=(12,30), gamma=(30,100))
id_num = 1

ch_names = ['RAW_AF7', 'RAW_AF8']
ch_types = ['eeg', 'eeg']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)
# read the relevant data from Muse results 
df_data = read_data(files_eeg_b[id_num-1])
# calculate Power Spectral Density 
df_data = df_data.transpose()
raw = mne.io.RawArray(df_data, info)
raw.filter(l_freq = 1, h_freq = 100, picks = 'eeg') #to check if needed- method = 'iir'
print(raw)
#raw.notch_filter(freqs=50, picks = 'eeg') - to check if needed
psd = raw.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8'])
#psd.plot()
psd_df = psd.to_data_frame()
#filter to frequancies between 1-100Hz
psd_df = psd_df[(psd_df["freq"]<=100) & (psd_df["freq"]>=1)]
psd_df.to_csv("psd_df.csv")

# calculate frequancies bands
freq_bands = dict(delta=(0.1,4), theta=(4,7), alpha=(8,15), beta=(16,31), gamma=(32,100))
columns = [('RAW_AF7', 'delta'), ('RAW_AF7', 'theta'), ('RAW_AF7', 'alpha'), ('RAW_AF7', 'beta'), ('RAW_AF7', 'gamma'), \
          ('RAW_AF8', 'delta'), ('RAW_AF8', 'theta'), ('RAW_AF8', 'alpha'), ('RAW_AF8', 'beta'), ('RAW_AF8', 'gamma')]


subject_data={}
i=0
for wave, tuple_range in freq_bands.items():
    wave_band_RAW_AF7 = (psd_df[(psd_df['freq'] >= tuple_range[0]) & (psd_df['freq'] <= tuple_range[1])]['RAW_AF7']).mean()
    subject_data[columns[i]] = [wave_band_RAW_AF7]
    i += 1
for wave, tuple_range in freq_bands.items():
    wave_band_RAW_AF8 = (psd_df[(psd_df['freq'] >= tuple_range[0]) & (psd_df['freq'] <= tuple_range[1])]['RAW_AF8']).mean()
    subject_data[columns[i]] = [wave_band_RAW_AF8]
    i += 1
df_subject_data = pd.DataFrame(subject_data)
df_subject_data.columns = pd.MultiIndex.from_tuples(df_subject_data.columns, names=['channel','wave'])
df_subject_data['subject'] = id_num
#df_subject_data.to_csv("df_subject_data.csv")

# add subject payment method:
questionarie = pd.read_excel(r"C:\Users\yehuda\pain_of_payment\Pain of Payment Questionnaire.xlsx",\
                             usecols = ['Payment_method'], sheet_name='Sheet0')
questionarie.drop(0, inplace=True)

df_subject_data['Payment_method'] = questionarie['Payment_method'].loc[id_num]

Creating RawArray with float64 data, n_channels=2, n_times=2832
    Range : 0 ... 2831 =      0.000 ...    11.059 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 100.00 Hz: -6.02, -6.02 dB

<RawArray | 2 x 2832 (11.1 s), ~52 kB, data loaded>
Effective window size : 1.000 (s)


In [25]:
'''
reading EEG measurements from time of buy +- 5 seconds 
using only the data from RAW_AF7 and RAW_AF8 electtrodes
'''
def read_data(file):
    
    # read data drom file
    df_data = pd.read_csv(file, low_memory=False, usecols = ['TimeStamp','RAW_AF7','RAW_AF8','ToB'])
    
    # adjust the time frame to be +- 5 seconds from time of buy
    ToB = pd.to_timedelta(df_data["ToB"].loc[0])
    before_5Sec = ToB - pd.to_timedelta("0:0:05")
    after_5Sec = ToB + pd.to_timedelta("0:0:05")
    df_data["TimeStamp"] = pd.to_timedelta(df_data["TimeStamp"])
    df_data = df_data[(df_data["TimeStamp"] <= after_5Sec) & (df_data["TimeStamp"] >= before_5Sec)]
    
    # use only the channels series (RAW_AF7,RAW_AF8) and only valid ones (not Nan)
    df_data.drop(["ToB", "TimeStamp"], inplace = True, axis=1)
    df_data.dropna(inplace=True)
    
    #optional - to save to csv
    #df_data.to_csv("df_data.csv",index=False)
    
    return df_data

In [26]:
'''
Analyzing the EEG data:
calculate Power Spectral Density
'''
def PSD(df_data, info):
    df_data = df_data.transpose()
    raw = mne.io.RawArray(df_data, info)
    raw.filter(l_freq = 1, h_freq = 100, picks = 'eeg')
    
    # calculate Power Spectral Density 
    psd = raw.compute_psd(method = 'welch', picks = ['RAW_AF7', 'RAW_AF8']) #to check if needed- method = 'iir'
    #raw.notch_filter(freqs=50, picks = 'eeg') - to check if needed
    
    # optional - to plot
    #psd.plot()
    
    psd_df = psd.to_data_frame()
    
    #filter to frequancies between 1-100Hz
    psd_df = psd_df[(psd_df["freq"]<=100) & (psd_df["freq"]>=1)]
    
    #optional - to save to csv
    #psd_df.to_csv("psd_df.csv")
    
    return psd_df

In [27]:
'''
calculate the mean of each wave band: delta, theta, alpha,beta, gama 
'''
def wave_bands(psd_df, bands, num_subject):
    
    columns = [('RAW_AF7', 'delta'), ('RAW_AF7', 'theta'), ('RAW_AF7', 'alpha'), ('RAW_AF7', 'beta'), ('RAW_AF7', 'gamma'), \
          ('RAW_AF8', 'delta'), ('RAW_AF8', 'theta'), ('RAW_AF8', 'alpha'), ('RAW_AF8', 'beta'), ('RAW_AF8', 'gamma')]
    subject_data={}
    i=0
    
    for wave, wave_range in bands.items():
        wave_band_RAW_AF7 = (psd_df[(psd_df['freq'] >= wave_range[0]) & (psd_df['freq'] <= wave_range[1])]['RAW_AF7']).mean()
        subject_data[columns[i]] = [wave_band_RAW_AF7]
        i += 1
    for wave, wave_range in freq_bands.items():
        wave_band_RAW_AF8 = (psd_df[(psd_df['freq'] >= wave_range[0]) & (psd_df['freq'] <= wave_range[1])]['RAW_AF8']).mean()
        subject_data[columns[i]] = [wave_band_RAW_AF8]
        i += 1
        
    df_subject_data = pd.DataFrame(subject_data)
    
    # rearnge the DataFrame so that each channel (RAW_AF7, RAW_AF8) has its own seperation to waves
    df_subject_data.columns = pd.MultiIndex.from_tuples(df_subject_data.columns, names=['channel','wave'])
    
    df_subject_data['subject'] = num_subject
    
    #optional - to save to csv
    #df_subject_data.to_csv("df_subject_data.csv")
    
    return df_subject_data

In [28]:
'''
main function
proccessing to each subject its EEG data from the ToB:
1. the mean of each band wave
2. the payment method (takes the answer from the questionarie)
'''
# initilaize variables
num_subject = 1
df_all_subjects_data = pd.DataFrame()
freq_bands = dict(delta=(0.1,4), theta=(4,7), alpha=(8,15), beta=(16,31), gamma=(32,100))

# read the questionarie payment information
questionarie = pd.read_excel(r"C:\Users\yehuda\pain_of_payment\Pain of Payment Questionnaire.xlsx",\
                             usecols = ['Payment_method'], sheet_name='Sheet0')
questionarie.drop(0, inplace=True)

# create info about the EEG expieremrnt
ch_names = ['RAW_AF7', 'RAW_AF8']
ch_types = ['eeg', 'eeg']
sfreq = 256
info = mne.create_info(ch_names = ch_names, ch_types=ch_types, sfreq = sfreq)

for file in files_eeg_b:
        
    # read the relevant data from Muse results 
    df_data = read_data(file)
    
    # calculate Power Spectral Density 
    df_PSD = PSD(df_data, info)
    
    # seperate to frequacies bands (waves) and calculate the mean for each wave
    df_subject_data = wave_bands(df_PSD, freq_bands, num_subject)
    
    #add the payment method of the subject
    df_subject_data['Payment_method'] = questionarie['Payment_method'].loc[num_subject]
    
    # add current subject data to all the data that was collected so far
    df_all_subjects_data = pd.concat([df_all_subjects_data, df_subject_data])
    
    num_subject += 1

#optional - to save to csv
df_all_subjects_data.to_csv("df_all_subjects_data.csv", encoding = 'utf-8-sig')

Creating RawArray with float64 data, n_channels=2, n_times=2832
    Range : 0 ... 2831 =      0.000 ...    11.059 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (3.301 sec)

Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=2, n_times=2808
    Range : 0 ... 2807 =      0.000 ...    10.965 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=2, n_times=2820
    Range : 0 ... 2819 =      0.000 ...    11.012 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (3.301 sec)

Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=2, n_times=2808
    Range : 0 ... 2807 =      0.000 ...    10.965 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=2, n_times=2820
    Range : 0 ... 2819 =      0.000 ...    11.012 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (3.301 sec)

Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=2, n_times=2820
    Range : 0 ... 2819 =      0.000 ...    11.012 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a 

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


    Range : 0 ... 2807 =      0.000 ...    10.965 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (3.301 sec)

Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=2, n_times=2820
    Range : 0 ... 2819 =      0.000 ...    11.012 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed tim

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


Creating RawArray with float64 data, n_channels=2, n_times=2832
    Range : 0 ... 2831 =      0.000 ...    11.059 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (3.301 sec)

Effective window size : 1.000 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


In [42]:
'''
calculate the mean power of each wave-band as a function of payment method
and display the results
'''
df_PSD_based_payment = df_all_subjects_data.groupby('Payment_method').mean().drop(labels='subject', level=0, axis=1)
df_PSD_based_payment

channel            RAW_AF7                                                \
wave                 delta       theta      alpha       beta       gamma   
Payment_method                                                             
אשראי           369.552312   95.861649  36.072312  32.047635  133.639753   
מזומן           680.617622  148.010388  46.010918  15.627317   10.503377   
סמארטפון        276.275248   40.481688  13.322975  12.782439   15.078595   

channel             RAW_AF8                                                 
wave                  delta       theta       alpha       beta       gamma  
Payment_method                                                              
אשראי            352.506206   99.677106   35.942681  28.581769  123.746109  
מזומן           2301.272940  407.072596  101.215634  31.729255  120.560395  
סמארטפון          78.342269   21.459219    8.715654   9.814229    6.156735